# Setup & Bibliotheken

In [ ]:
'''# Schritt 1: Deinstalliere alle problematischen Pakete
!pip uninstall tensorflow tensorflow-text tf-keras tensorflow-decision-forests tensorflow-probability -y

# Schritt 2: Installiere kompatible Versionen
!pip install tensorflow==2.19.0 tensorflow-probability==0.25.0 tensorflow-text==2.19.0 tf-keras==2.19.0

!pip install -q sentence-transformers transformers accelerate torch pandas tqdm

# Schritt 3: Unterdrücke Warnungen (für parakeet und CUDA)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Unterdrückt TensorFlow-Warnungen

# Schritt 4: Teste den Import
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
print("Import erfolgreich!")

# Schritt 5: Teste tensorflow_probability
import tensorflow_probability as tfp
print(f"TensorFlow Probability Version: {tfp.__version__}")
!pip install transformers torch pandas tqdm -q'''

'# Schritt 1: Deinstalliere alle problematischen Pakete\n!pip uninstall tensorflow tensorflow-text tf-keras tensorflow-decision-forests tensorflow-probability -y\n\n# Schritt 2: Installiere kompatible Versionen\n!pip install tensorflow==2.19.0 tensorflow-probability==0.25.0 tensorflow-text==2.19.0 tf-keras==2.19.0\n\n!pip install -q sentence-transformers transformers accelerate torch pandas tqdm\n\n# Schritt 3: Unterdrücke Warnungen (für parakeet und CUDA)\nimport os\nos.environ[\'TF_CPP_MIN_LOG_LEVEL\'] = \'2\'  # Unterdrückt TensorFlow-Warnungen\n\n# Schritt 4: Teste den Import\nfrom transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline\nprint("Import erfolgreich!")\n\n# Schritt 5: Teste tensorflow_probability\nimport tensorflow_probability as tfp\nprint(f"TensorFlow Probability Version: {tfp.__version__}")\n!pip install transformers torch pandas tqdm -q'

In [33]:

import platform #infos aus dem System verarbeiten können (Windows)
import os #für Dateihändelung
import pandas as pd #für Datenanalyse (Bereinigung von Daten)
import random
from tqdm import tqdm #um ausgeben zu können, wie weit die Prozesse laufen (für Prozess Balken)
import tensorflow_probability as tfp # Wahrscheinlichekti von Ergebnissen analysieren & bewerten
import torch # soll dabei helfen, schnellere Berechnungen zu machen (GPU)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM #aus hugging face bibliothek, um Texte in Tokens umzuwandeln, Textsequenzen als Input und andere Textsequenzen ausgeben (Output)
#Aufgabe: Übersetzen, Zusammenfassen, Generieren
from sentence_transformers import SentenceTransformer, util #sentenceTranformer, damit werden Modelle geladen, die Texte in ganze Vektoren umwandeln
#ür Sätze/Texte erzeugen → Ähnlichkeitsvergleiche
from google.colab import files, drive #Interaktion mit lokalen Dateien in Google Colab Umgebung
from datetime import datetime# Manipulation von Zeitangaben
import shutil #Operationen mit Dateien und mit dem Sammeln von Dateien
import re #Operationen für reguläre Ausdrücke (regex)
import json
from nltk.tokenize import sent_tokenize
import nltk
# NLTK Punkt-Tokenizer
nltk.download('punkt', quiet=True)
import math
import csv
from pathlib import Path
import requests
from dotenv import load_dotenv




In [3]:
load_dotenv()
API_KEY = os.getenv("GROK_API_KEY")

if not API_KEY:
    raise ValueError("Set GROK_API_KEY in .env")

print("API key loaded securely")

API key loaded securely


In [4]:
%%bash
echo ".env" >> .gitignore

In [5]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

# Gewählte xlsl Datei in csv Datei umwandeln und umbenennen

In [ ]:
# @title

read_file = pd.DataFrame(pd.read_excel('/content/drive/My Drive/Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).xlsx'))
read_file.to_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv", index = False, header = True, encoding=smartEncoding())
df = pd.DataFrame(pd.read_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv"))
#print(df.head)

In [ ]:
# @title
#wähle relevante Spalten
gewaehlte_spalten = df[["Erfahrungsbericht des Nutzers" , "Zufallszahl"]]

#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv", index = False)




# Datenbereinigung

In [ ]:
erfahrungen_gefiltert = pd.read_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_gefiltert.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_gefiltert.drop_duplicates(inplace=True)
#speichern
erfahrungen_gefiltert.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index=False)# Entfernung von Duplikaten wurde beim df selbst vollzogen anstatt als Kopie
print(f"FINAL: {len(erfahrungen_gefiltert)} Reihen")


#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)


#zufällig n = 100 Berichte für erfahrungen_final.csv auswählen
erfahrungen_clean = pd.read_csv("/content/drive/My Drive/erfahrungen_clean.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_clean.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_clean.drop_duplicates(inplace=True)
erfahrungen_clean.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)

# INDEX-Nummern zufällig wählen
zufalls_index = random.sample(range(len(erfahrungen_clean)), k=100)
zufall_100 = erfahrungen_clean.iloc[zufalls_index]



#speichern (NUR 100!)
zufall_100.to_csv("/content/drive/My Drive/zufall_100_berichte.csv", index=False)

print(f"{len(zufall_100)} Zufallsberichte!")

FINAL: 6453 Reihen
100 Zufallsberichte!


# **Datenaugmentation und Syn-Chain-Methode**

## Datenaugmentation mit Trainingsdaten (ca. 11.900 Rezensionen)

## Paraphrasieren

In [6]:

# Google Drive mounten/ Ordner in Google Colab Umbegung erstellen lassen (einmalige Ausführung nötig)
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/Projekt_ABSA' #Unterordner "Projekt_ABSA" in Google Colab Umgebung erstellen
os.makedirs(SAVE_PATH, exist_ok=True) #Methode, wobei ein Projektverzeichnis (Pfad für Paraphrase mit inbegriffen) festgelegt wird

print("Ordner & Bibliotheken geladen!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ordner & Bibliotheken geladen!


sieht nice aus

In [32]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

# Erstellung eines "Synonym-Dictionarys", zuerste mit Schlüsselwert wie z.B. "schmutzig" & danach Wertpaaren
SYNONYMS = {
    "Personal": ["Team", "Mitarbeiter", "Pflegekräfte", "Belegschaft", "Angestellte"],
    "Pflege": ["Betreuung", "Fürsorge", "Versorgung", "Hilfe", "Obhut"],
    "freundlich": ["nett", "höflich", "herzlich", "zuvorkommend", "liebenswürdig"],
    "unfreundlich": ["kalt", "respektlos", "rüde", "abweisend"],
    "schön": ["hübsch", "ansprechend", "gemütlich", "anheimelnd"],
    "gut": ["ordentlich", "zufriedenstellend", "positiv", "lobenswert"],
    "schlecht": ["mies", "mangelhaft", "unzureichend", "fragwürdig"],
    "Essen": ["Speisen", "Nahrung", "Mahlzeiten", "Gerichte"],
    "lecker": ["köstlich", "schmackhaft", "gut"],
    "sauber": ["gepflegt", "rein", "ordentlich", "hygienisch"],
    "hilfsbereit": ["unterstützend", "kooperativ", "engagiert", "aufmerksam"],
    "im": ["in", "bei"],
    "und": ["sowie", "–", "&"],
    "sehr": ["wirklich", "besonders", "äußerst"],
    "ist": ["bleibt", "erscheint"],
    "Haus": ["Heim", "Einrichtung"],
    "Heim": ["Haus", "Pflegeheim", "Institution", "Residenz"],
    "Mutter": ["Familienangehörige"],
    "Oma": ["Großmutter", "Omi"],
    "Opa": ["Großvater"],
    "kompetent": ["professionell", "fähig", "tüchtig"],
    "nett": ["sympathisch", "herzlich", "angenehm"],
    "Zimmer": ["Räume", "Wohnbereich", "Appartement"],
    "Bewohner": ["Senioren", "Heimbewohner", "Pflegebedürftige"],
    "Einrichtung": ["Institution", "Haus", "Pflegeheim"],
    "motiviert": ["engagiert", "eifrig", "tatkräftig"],
    "zugewandt": ["aufmerksam", "herzlich", "fürsorglich"],
    "Kommunikation": ["Austausch", "Kontakt", "Gespräche"],
    "Angehörigen": ["Familienangehörigen", "Verwandten"],
    "liebevoll": ["herzlich", "zärtlich", "fürsorglich"],
    "gepflegt": ["versorgt", "betreut", "umhegt"],
    "muss": ["soll", "hat zu sein"],
    "wenn": ["falls", "sobald"],
    "irgendetwas": ["etwas", "manchmal etwas"],
    "vermeintlich": ["angeblich", "vermeintlich", "scheinbar"],
    "nicht": ["nicht", "nicht ganz"],
    "läuft": ["funktioniert", "klappt"],
    "kann": ["darf", "ist möglich"],
    "jederzeit": ["immer", "sofort", "jederzeit"],
    "ansprechen": ["erwähnen", "anbringen", "zur Sprache bringen"],
    "gemeinsame": ["zusammen", "gemeinsam gefundene"],
    "Lösung": ["Antwort", "Ausweg"],
    "finden": ["erreichen", "entdecken"],
}

CANONICAL = {}
for key, syns in SYNONYMS.items():
    key_lower = key.lower()
    CANONICAL[key_lower] = key_lower
    for s in syns:
        CANONICAL[s.lower()] = key_lower

#Synonyme ersetzen lassen
def synonym_replace(text):
    result = text
    words = set(re.findall(r'\b\w+\b', text.lower()))
    replaced = set()
    for word in words:
        if word in CANONICAL and word not in replaced:
            main = CANONICAL[word]
            candidates = [s for s in SYNONYMS.get(main.capitalize(), []) if s.lower() != word]
            if candidates:
                repl = random.choice(candidates)
                pattern = re.compile(rf'\b{re.escape(word)}\b', re.IGNORECASE)
                result = pattern.sub(
                    lambda m: repl.capitalize() if m.group(0)[0].isupper() else repl,
                    result,
                    count=1
                )
                replaced.add(word)
    return result

#Satz umstrukturieren
def restructure_sentences(text):
    if len(text) < 100:
        return text
    sents = sent_tokenize(text)
    if len(sents) <= 1:
        return text
    if random.random() < 0.75:
        if len(sents) == 2:
            return sents[1] + " " + sents[0]
        else:
            first, last = sents[0], sents[-1]
            middle = sents[1:-1]
            if middle:
                random.shuffle(middle)
            return first + " " + " ".join(middle) + " " + last
    return text

#pro Rezension 3 Paraphrasen erstellen
def generate_paraphrases(text):
    original = text.strip()
    if len(original) < 20:
        return [original] * 3

    results = set()
    attempts = 0
    max_attempts = 80

    while len(results) < 3 and attempts < max_attempts:
        step1 = synonym_replace(original)
        if len(step1) > 80 and random.random() < 0.65:
            para = restructure_sentences(step1)
        else:
            para = step1

        if para != original and len(para) > len(original) * 0.6:
            results.add(para)
        attempts += 1

    result_list = list(results)
    while len(result_list) < 3:
        result_list.append(original)
    return result_list[:3]

#Daten aus "erfahrungen_clean.csv hochladen"
input_path = "/content/drive/MyDrive/erfahrungen_clean.csv"
df = pd.read_csv(input_path)

# Sicherstellen, dass Spalte existiert
if 'Erfahrungsbericht des Nutzers' not in df.columns:
    raise ValueError("Spalte 'Erfahrungsbericht des Nutzers' nicht gefunden!")

df = df.dropna(subset=['Erfahrungsbericht des Nutzers']).reset_index(drop=True)
df = df.reset_index().rename(columns={'index': 'id'})

print(f"Geladene Rezensionen: {len(df)}")

#in JSONL Datei speichern
output_path = "/content/drive/MyDrive/Projekt_ABSA/erfahrungen_paraphrased.jsonl"

with open(output_path, 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        original_id = int(row['id'])
        original_text = str(row['Erfahrungsbericht des Nutzers'])

        # 1. Originalzeile
        original_entry = {
            "id": original_id,
            "original": original_text
        }
        f.write(json.dumps(original_entry, ensure_ascii=False) + "\n")

        # 2. 3 Paraphrasen pro Rezension
        paraphrases = generate_paraphrases(original_text)
        for i, para in enumerate(paraphrases, 1):
            para_entry = {
                "variant_id": i,
                "paraphrase": para
            }
            f.write(json.dumps(para_entry, ensure_ascii=False) + "\n")

print(f"\nFERTIG! JSONL gespeichert:")
print(f"   → {output_path}")
print(f"   → {len(df)} Originale + {len(df)*3} Paraphrasen = {len(df)*4} Zeilen")

#download der paraphrasierten Rezensionen
from google.colab import files
print("\nStarte Download...")
files.download(output_path)

Geladene Rezensionen: 6453

FERTIG! JSONL gespeichert:
   → /content/drive/MyDrive/Projekt_ABSA/erfahrungen_paraphrased.jsonl
   → 6453 Originale + 19359 Paraphrasen = 25812 Zeilen

Starte Download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Nan-Werte entfernen aus JSONL-Datei

In [ ]:

input_file = "/content/sample_data/data_training.jsonl"
output_file = "cleaned_jsonl.jsonl"

def find_nan(obj):
    if isinstance(obj, float):
        return math.isnan(obj) or math.isinf(obj)
    if isinstance(obj, dict):
        return any(find_nan(v) for v in obj.values())
    if isinstance(obj, list):
        return any(find_nan(v) for v in obj)
    return False

with open(input_file, 'r', encoding=smartEncoding()) as f, \
     open(output_file, 'w', encoding=smartEncoding()) as fi:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            if not find_nan(obj):
                fi.write(json.dumps(obj, ensure_ascii=False) + '\n')
        except json.JSONDecodeError:
            pass  # skip invalid lines


#**Trainingsdaten und Testdaten aufteilen**

# **Anwendung der Syn-Chain-Methode** - beim Fine Tuning

In [ ]:
#/content/drive/MyDrive/cleaned_jsonl_training.jsonl, daran weiter arbeiten



für Fine Tuning:
Hugging Face: Fine-tune ein Grok-ähnliches Modell (z. B. "xai-org/grok-1" oder "reedmayhew/Grok-3-gemma3-4B-distilled" als Distillation von Grok 3). Das ist open-source und GitHub-freundlich.

Code-Beispiel (Distillation von Grok 3 via API + Fine-Tuning auf Gemma-3 4B):


Best Practices (aus Reddit, GitHub Blog, HF Docs):

Lizenz: Füge eine LICENSE (z. B. Apache 2.0) hinzu, da Grok-Modelle proprietär sind – dein Fine-Tuning erbt das.
Model Card: Erstelle eine (HF-Template: https://huggingface.co/docs/hub/model-cards) mit Details zu Daten (12.000 Rezensionen), Augmentation und Metriken.
Datenschutz: Anonymisiere sensible Daten in der CSV (z. B. Namen entfernen), bevor du hochlädst – Pflegeberichte könnten personenbezogen sein (DSGVO-konform!).
Versionskontrolle: Nutze GitHub Releases für große Dateien (z. B. Weights als Asset).
Integration mit HF: Push zu Hugging Face Hub und verlinke im GitHub-Repo – HF ist GitHub-ähnlich und unterstützt Grok-Destillationen (z. B. "reedmayhew/Grok-3-gemma3-4B-distilled").



3. Empfehlung für deine Bachelorarbeit

Starte mit Option A/B: Prompt Engineering für schnelle Tests, dann Hugging Face für echtes Fine-Tuning (ca. 30–60 Min. in Colab mit GPU).
GitHub-Repo als Portfolio: Lade Code, Skripte und Metriken hoch – das zeigt Reproduzierbarkeit. Vermeide rohe Weights (>100 MB) direkt; verlinke sie.
Nächste Schritte: Hole dir einen xAI API-Key und HF-Token. Teste den Code in Colab.

Falls du Hilfe beim Setup (z. B. API-Key, spezifischer Code-Fehler) brauchst oder mehr Details zu einer Option, lass es mich wissen!

In [ ]:
# um damit zu beginnen: https://arxiv.org/html/2507.09485